In [14]:
import os
from langchain_core.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_community.llms import CTransformers
from langchain_pinecone import PineconeVectorStore
from dotenv import load_dotenv

In [26]:
load_dotenv()

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
index_name = "medical-chatbot"

In [16]:
# Extract data from PDF file.
def load_pdf(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents


In [17]:
extracted_data = load_pdf("../data")


In [18]:
# Document 5
extracted_data[5]


Document(page_content='The Gale Encyclopedia of Medicine 2 is a medical ref-\nerence product designed to inform and educate readersabout a wide variety of disorders, conditions, treatments,and diagnostic tests. The Gale Group believes the productto be comprehensive, but not necessarily definitive. It isintended to supplement, not replace, consultation with aphysician or other healthcare practitioner. While the GaleGroup has made substantial efforts to provide informationthat is accurate, comprehensive, and up-to-date, the GaleGroup makes no representations or warranties of anykind, including without limitation, warranties of mer-\nchantability or fitness for a particular purpose, nor does itguarantee the accuracy, comprehensiveness, or timelinessof the information contained in this product. Readersshould be aware that the universe of medical knowledgeis constantly growing and changing, and that differencesof medical opinion exist among authorities. Readers arealso advised to seek profe

### Chunks Creation

In [19]:
# Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks


In [20]:
text_chunks = text_split(extracted_data=extracted_data)
print("Length of my chunk:", len(text_chunks))

Length of my chunk: 7020


In [21]:
text_chunks[5]

Document(page_content='mation presented in this publication, the Gale Group neither guaranteesthe accuracy of the data contained herein nor assumes any responsibili-ty for errors, omissions or discrepancies. The Gale Group accepts nopayment for listing, and inclusion in the publication of any organiza-tion, agency, institution, publication, service, or individual does notimply endorsement of the editor or publisher. Errors brought to theattention of the publisher and verified to the satisfaction of the publish-er', metadata={'source': '..\\data\\Medical_book.pdf', 'page': 3})

In [22]:
# Download embedding model
def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )

    return embeddings


In [23]:
embeddings = download_hugging_face_embedding()


In [24]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))


Length 384


In [25]:
query_result

[-0.03447718918323517,
 0.031023189425468445,
 0.006734984926879406,
 0.026109009981155396,
 -0.03936197981238365,
 -0.16030246019363403,
 0.06692398339509964,
 -0.006441500969231129,
 -0.04745054990053177,
 0.01475889328867197,
 0.07087533175945282,
 0.05552756413817406,
 0.01919332891702652,
 -0.02625131793320179,
 -0.01010945811867714,
 -0.026940524578094482,
 0.022307483479380608,
 -0.022226670756936073,
 -0.14969268441200256,
 -0.017493031919002533,
 0.007676261011511087,
 0.05435234680771828,
 0.0032544753048568964,
 0.03172600269317627,
 -0.0846213549375534,
 -0.029405953362584114,
 0.05159570649266243,
 0.0481240414083004,
 -0.003314815927296877,
 -0.05827920883893967,
 0.04196928068995476,
 0.022210700437426567,
 0.1281888484954834,
 -0.02233891189098358,
 -0.011656275950372219,
 0.06292835623025894,
 -0.032876234501600266,
 -0.09122606366872787,
 -0.031175419688224792,
 0.05269957706332207,
 0.0470348596572876,
 -0.08420299738645554,
 -0.030056182295084,
 -0.02074472792446613

In [27]:
#Initializing the Pinecone
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
vectorstore.from_texts([t.page_content for t in text_chunks], index_name=index_name, embedding=embeddings)


In [28]:
#If we already have an index we can load it like this
query = "What are Allergies"

docsearch=vectorstore.similarity_search(query, k=3)

docsearch

[Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"),
 Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives\nThe following types of drugs commonly cause al

In [29]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [30]:
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [32]:
MODEL_PATH = "../model/llama-2-7b-chat.ggmlv3.q4_0.bin"

llm = CTransformers(
    model=MODEL_PATH,
    model_type="llama",
    config={
        "max_new_tokens": 512,
        "temperature": 0.8,
    },
)

In [33]:
# Initialize the RetrievalQA chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = vectorstore.as_retriever(search_kwargs={"k":2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

In [34]:
while True:
    user_input = input(f"How can i help you today: ")
    result=qa.invoke({"query": user_input})
    print(result["result"])
    break

Acne is a skin disease that occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria. It can affect various parts of the body, including the face, chest, and back.
